In [1]:
# Install packages
# %pip install accelerate click einops einops_exts ema-pytorch fsspec kornia numpy packaging pillow pydantic pytorch-lightning pytorch-warmup sentencepiece torch torchvision transformers tqdm --quiet

In [2]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive/')

# # Copy imagen folder locally
# !cp -r /content/drive/MyDrive/imagen .

In [3]:
# %pip install imagen_pytorch --quiet

In [4]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer

/home/adiel/anaconda3/envs/imagen/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
torch.cuda.empty_cache()

In [6]:
# %pip install import-ipynb --quiet

In [7]:
import import_ipynb

from COCODataset import COCODataset

importing Jupyter notebook from COCODataset.ipynb


In [8]:
# unet for imagen

unet1 = Unet(
    dim = 32,
    cond_dim = 128,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
)

# unet2 = Unet(
    # dim = 32,
    # cond_dim = 128,
    # dim_mults = (1, 2, 4, 8),
    # num_resnet_blocks = (2, 4, 8, 8),
    # layer_attns = (False, False, False, True),
    # layer_cross_attns = (False, False, False, True)
# )

The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/


In [9]:
# imagen, which contains the unets above (base unet and super resoluting ones)
imagen = Imagen(
    unets = unet1,
    text_encoder_name = 't5-base',
    image_sizes = 32,
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

In [10]:
# wrap imagen with the trainer class
trainer = ImagenTrainer(
    imagen, 
    split_valid_from_train = True,
    checkpoint_path = "./cp/",
    checkpoint_every=5000
).cuda()

no checkpoints found to load from at ./cp/


In [11]:
dataset = COCODataset('./data/train2017/train2017', './embeddings', './annotations', "./paths/im_paths.json", "./paths/em_paths.json", image_size = 32, max_embed_size=32, save_paths=False)

In [12]:
print(len(dataset))

236936


In [13]:
print(len(list(dataset.id_to_empath.keys())))

236936


In [14]:
trainer.add_train_dataset(dataset, batch_size = 64)

training with dataset of 231012 samples and validating with randomly splitted 5924 samples


In [15]:
# working training loop
for i in range(200000):
    loss = trainer.train_step(unet_number = 1, max_batch_size = 64)

    print(f'{i}: loss: {loss}')

    # if not (i % 50):
    #     valid_loss = trainer.valid_step(unet_number = 1, max_batch_size = 4)
    #     print(f'valid loss: {valid_loss}')

    if not ((i+1) % 5000) and trainer.is_main: # is_main makes sure this can run in distributed
        images = trainer.sample(texts = [
            'A red school bus parked in a parking lot',
            'A blue school bus parked in a parking lot',
            'The decadent chocolate dessert is on the table',
            'A bowl of bananas is on the table'
        ], cond_scale = 3., return_pil_images = True)
        images[0].save(f'./out/sample-redbus-{i // 1000}.png')
        images[1].save(f'./out/sample-bluebus-{i // 1000}.png')
        images[2].save(f'./out/sample-chocolate-{i // 1000}.png')
        images[3].save(f'./out/sample-bananas-{i // 1000}.png')

        # trainer.save(f'./cp/1-{i}.pt')

0: loss: 0.6646080017089844
1: loss: 0.6407820582389832
2: loss: 0.6542752981185913
3: loss: 0.6021029949188232
4: loss: 0.6461031436920166
5: loss: 0.6747523546218872
6: loss: 0.648411750793457
7: loss: 0.5932953357696533
8: loss: 0.623455822467804
9: loss: 0.601704478263855
10: loss: 0.6480996012687683
11: loss: 0.6318804621696472
12: loss: 0.5791606903076172
13: loss: 0.631140410900116
14: loss: 0.6463558077812195
15: loss: 0.6028473377227783
16: loss: 0.6320886611938477
17: loss: 0.6787024736404419
18: loss: 0.6400778293609619
19: loss: 0.6525121331214905
20: loss: 0.6381585597991943
21: loss: 0.6270104646682739
22: loss: 0.6225738525390625
23: loss: 0.647109866142273
24: loss: 0.6049071550369263
25: loss: 0.6363463401794434
26: loss: 0.620418131351471
27: loss: 0.6227573156356812
28: loss: 0.6283382773399353
29: loss: 0.6467706561088562
30: loss: 0.6446770429611206
31: loss: 0.6330771446228027
32: loss: 0.6462870836257935
33: loss: 0.6344558000564575
34: loss: 0.6142666935920715
3

sampling loop time step: 100%|██████████| 1000/1000 [01:11<00:00, 14.08it/s]
1it [01:11, 71.09s/it]


RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 7.79 GiB total capacity; 5.56 GiB already allocated; 260.44 MiB free; 6.02 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF